In [1]:
#引入需要的包裝

import pyvisa
import time
import math

### RF-150A_100D

===============Function List=================

* Set power state
* Set gain magnitude

控制前要將撥動開關往上撥到REMOTE，儀器才會接收指令

In [2]:
class RF_150A100D():
    def __init__(self, resource_address, visa_dll=None):
        """
        初始化儀器控制器。
        :param resource_address: 儀器的資源地址。
        :param visa_dll: 使用的 VISA DLL 路徑。如果為 None，則使用系統預設空白。
        """
        self.resource_address = resource_address
        self.visa_dll = visa_dll if visa_dll is not None else ''
        self.rm = pyvisa.ResourceManager(self.visa_dll)
        self.instrument = None
        self.connect_to_instrument()
        
        
    def connect_to_instrument(self):
        # 建立與儀器的連接
        self.instrument = self.rm.open_resource(self.resource_address)
        print("Connected to instrument.")
        print(self.query_identity())

    def query_identity(self):
         # 查詢並返回儀器的身份識別信息
        if self.instrument:
             return self.instrument.query("*IDN?")
        else:
             raise Exception("Instrument not connected. Please connect first.")

    def close_instrument(self):
        if self.instrument:
            self.instrument.close()
            print("Instrument connection closed.")
        else:
            print("No instrument to close.")

    def power_on(self):
        if self.instrument:
            self.instrument.write(f"P1")
            print(f"turn power on")
        else:
            raise Exception("Instrument not connected. Please connect first.")

    def power_off(self):
        if self.instrument:
            self.instrument.write(f"P0")
            print(f"turn power off")
        else:
            raise Exception("Instrument not connected. Please connect first.")

    def set_gain(self,gain ,gain_set):
        if self.instrument:
            self.instrument.write(f"G{gain_set}")
            print(f"set gain be G{gain}")
        else:
            raise Exception("Instrument not connected. Please connect first.")

    def reset(self):
        if self.instrument:
            self.instrument.write(f"R")
            print(f"clear faults")
        else:
            raise Exception("Instrument not connected. Please connect first.")

In [3]:
def calculate_gain_set(gain):
    """
    計算並返回設置增益的值
    透過設定的gain去計算給儀器的gain_set該為多少
    """
    ratio = 4095 / 100  # MAX和MIN的比例
    gain_set = math.ceil(ratio * gain)  # 使用 math函數進行無條件進位
    gain_set = min(gain_set, 4095)  # 限制 gain_set 最大值為 4095
    gain_set = str(gain_set).zfill(4)  # 保持四個字元
    print(ratio * gain)
    return gain_set

In [4]:
def main():
    
    # =========================Parmeters===============================
    """
    Gain : 0~100
    Power state : { ON | OFF }
    """
    gain = 5
    power_state = 'OFF'
    
    #====================Instrument Connect Test========================
    rf_instrument_address = 'USB0::0x0547::0x1B58::0358752::INSTR'
    rf = RF_150A100D(rf_instrument_address)
     #若有其他儀器串接下去...

    #============================Auto Test==============================
    # TO DO:寫For迴圈自動測量的區域
    
    gain_set = calculate_gain_set(gain)  # 計算增益設置值
    rf.set_gain(gain, gain_set)

    if power_state == 'ON': #判斷書出狀態，選擇def
        rf.power_on()
    else:
        rf.power_off()
    
    # ============================ Close Instrument======================
    rf.close_instrument()
    
if __name__ == "__main__":
    main()

Connected to instrument.
AR-RF/MICROWAVE-INST,150A100D,1.0

204.75
set gain be G5
turn power off
done all things
Instrument connection closed.
